В моей курсовой я использую датасеты на русском для задач анализа тональности и детекции токсичности. Я решила попробовать посчитать на них статистику, а именно: у меня есть идея, что токсичные тексты и тексты с негативной тональностью будут более похожи, чем все остальные. Возможно, это будет работать и в обратную сторону - не токсичные тексты и положительные, но мне кажется, что токсичности и негативность более маркированы.


**Итого имеем:**

Нулевая гипотеза: тексты с лэйблами Bad + 1 *будут не больше похожи*, чем тексты с лэйблами Bad + 0, Good + 1, Good + 0

Альтернативная гипотеза: тексты с лэйблами Bad + 1 будут *более похожи*, чем тексты с лэйблами Bad + 0, Good + 1, Good + 0


Про сами датасеты:

[Датасет для анализа тональности](https://huggingface.co/datasets/ai-forever/kinopoisk-sentiment-classification):
* Объем: 10500 текстов
* Соотношение классов: 3500:3500:3500
* Классы: Good, Bad, Neutral
* Состав: отзывы на фильмы из Кинопоиска

[Датасет для детекции токсичности](https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments):
* Объем: 14412 текста
* Соотношение классов: 4826:9586
* Классы: 0, 1
* Состав: комментарии из Двача и Пикабу

In [2]:
!pip install datasets==2.16.1 fsspec==2023.6.0
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 115.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import kagglehub
import pandas as pd
from datasets import load_dataset
from collections import Counter, defaultdict
import spacy
import multiprocessing
from tqdm import tqdm
from scipy.stats import chi2_contingency
import os

Импортирую датасеты

In [4]:
path = kagglehub.dataset_download("blackmoon/russian-language-toxic-comments")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/russian-language-toxic-comments


In [5]:
tox_raw = pd.read_csv('/root/.cache/kagglehub/datasets/blackmoon/russian-language-toxic-comments/versions/1/labeled.csv')
tox_raw = tox_raw

Этот датасет поделен на train, val и test по дефолту, нам трейна хватит

In [6]:
sent_raw = load_dataset("ai-forever/kinopoisk-sentiment-classification")
sent_raw = sent_raw['train']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Создаю частотные словари для сентимента

In [7]:
# Загружаем модель spaCy
nlp = spacy.load("ru_core_news_sm", disable=["ner", "parser"])
nlp.max_length = 2000000  # на всякий случай

# Предобработка одной пачки
def process_batch(batch):
    results = []
    docs = list(nlp.pipe(batch["text"], batch_size=64, n_process=multiprocessing.cpu_count()))
    for doc in docs:
        lemmas = [
            token.lemma_.lower()
            for token in doc
            if token.is_alpha and not token.is_stop
        ]
        results.append(lemmas)
    return {"lemmas": results}


# Применяем к датасету
ds_with_lemmas = sent_raw.map(
    process_batch,
    batched=True,
    batch_size=64,
    num_proc=multiprocessing.cpu_count(),
    desc="Лемматизация",
)

# Строим частотные словари по лейблам
label_freqs_sent = defaultdict(Counter)

for example in ds_with_lemmas:
    label = example["label_text"]
    label_freqs_sent[label].update(example["lemmas"])

# это я сохраняю результаты
output_dir = "freq_sent"
os.makedirs(output_dir, exist_ok=True)

for lbl, counter in label_freqs_sent.items():
    # Превращаем Counter в DataFrame
    df = pd.DataFrame(counter.items(), columns=["lemma", "frequency"])
    # Сортируем по убыванию частоты
    df = df.sort_values(by="frequency", ascending=False).reset_index(drop=True)
    # Сохраняем в CSV
    out_path = os.path.join(output_dir, f"freq_sent_label_{lbl}.csv")
    df.to_csv(out_path, index=False, encoding="utf-8")
    print(f"Сохранён файл: {out_path} (строк: {len(df)})")

# Пример: вывести топ-20 лемм
print("Топ лемм для класса Good:")
for lemma, freq in label_freqs_sent['Good'].most_common(20):
    print(f"{lemma}: {freq}")
print("Топ лемм для класса Bad:")
for lemma, freq in label_freqs_sent['Bad'].most_common(20):
    print(f"{lemma}: {freq}")
print("Топ лемм для класса Neutral:")
for lemma, freq in label_freqs_sent['Neutral'].most_common(20):
    print(f"{lemma}: {freq}")

Сохранён файл: freq_sent/freq_sent_label_Good.csv (строк: 42444)
Сохранён файл: freq_sent/freq_sent_label_Bad.csv (строк: 47433)
Сохранён файл: freq_sent/freq_sent_label_Neutral.csv (строк: 45814)
Топ лемм для класса Good:
фильм: 16533
человек: 5397
жизнь: 3633
хороший: 3277
герой: 2986
время: 2436
главный: 2419
актёр: 2415
смотреть: 2319
роль: 2309
картина: 2204
история: 2179
первый: 1990
сказать: 1986
кино: 1912
раз: 1828
мир: 1792
сюжет: 1773
год: 1740
любовь: 1686
Топ лемм для класса Bad:
фильм: 18639
хороший: 3374
человек: 3022
смотреть: 2674
актёр: 2643
герой: 2617
сюжет: 2475
кино: 2407
сказать: 2379
главный: 2266
первый: 2213
ни: 2206
время: 2123
игра: 2013
говорить: 1805
раз: 1735
картина: 1704
режиссёр: 1651
зритель: 1576
просмотр: 1575
Топ лемм для класса Neutral:
фильм: 15655
человек: 3570
хороший: 3107
герой: 2648
жизнь: 2258
главный: 2080
сказать: 2051
смотреть: 2037
актёр: 1962
первый: 1922
время: 1912
кино: 1853
сюжет: 1853
картина: 1681
роль: 1663
игра: 1640
ни: 1610
р

Пипипупу.

У положительных и негативных отзывов частотные слова практически одинаковые. Это странно, но у меня и в курсаче подозрительно низкое качество на нем выходило, я планировала проверить датасет. Вот и проверила:)

То же самое для тональности

In [8]:
batch_size = 64
# проверить равномерность распределения
# === ЗАГРУЗКА CSV ===
texts = tox_raw['comment'].tolist()
labels = tox_raw['toxic'].tolist()

In [9]:
# === ИНИЦИАЛИЗАЦИЯ spaCy ===
nlp = spacy.load("ru_core_news_sm", disable=["ner", "parser"])
nlp.max_length = 2_000_000

# === ХРАНИЛИЩЕ ДЛЯ ЧАСТОТ ===
label_freqs_tox = defaultdict(Counter)

# === ОБРАБОТКА БАТЧАМИ ===
def yield_batches(texts, labels, batch_size):
    for i in range(0, len(texts), batch_size):
        yield texts[i:i+batch_size], labels[i:i+batch_size]

for text_batch, label_batch in tqdm(yield_batches(texts, labels, batch_size), total=len(texts)//batch_size + 1):
    docs = list(nlp.pipe(text_batch, batch_size=64, n_process=multiprocessing.cpu_count()))
    for doc, label in zip(docs, label_batch):
        lemmas = [
            token.lemma_.lower()
            for token in doc
            if token.is_alpha and not token.is_stop
        ]
        label_freqs_tox[label].update(lemmas)

output_dir = "freq_tox"
os.makedirs(output_dir, exist_ok=True)

for lbl, counter in label_freqs_tox.items():
    # Превращаем Counter в DataFrame
    df = pd.DataFrame(counter.items(), columns=["lemma", "frequency"])
    # Сортируем по убыванию частоты
    df = df.sort_values(by="frequency", ascending=False).reset_index(drop=True)
    # Сохраняем в CSV
    out_path = os.path.join(output_dir, f"freq_tox_label_{lbl}.csv")
    df.to_csv(out_path, index=False, encoding="utf-8")
    print(f"Сохранён файл: {out_path} (строк: {len(df)})")

# === ВЫВОД ТОП-20 ЛЕММ ДЛЯ КАЖДОГО КЛАССА ===
for lbl, counter in label_freqs_tox.items():
    print(f"\nТоп-20 лемм для класса {lbl}:")
    for lemma, freq in counter.most_common(20):
        print(f"{lemma}: {freq}")

100%|██████████| 226/226 [03:00<00:00,  1.25it/s]

Сохранён файл: freq_tox/freq_tox_label_1.0.csv (строк: 19917)
Сохранён файл: freq_tox/freq_tox_label_0.0.csv (строк: 28584)

Топ-20 лемм для класса 1.0:
человек: 331
хохол: 241
год: 194
говорить: 181
хороший: 164
тупой: 155
русский: 155
раз: 155
тред: 142
знать: 142
делать: 135
нахуй: 129
смотреть: 124
думать: 121
россия: 120
видеть: 119
ни: 117
писать: 110
сделать: 109
дело: 109

Топ-20 лемм для класса 0.0:
год: 1311
человек: 786
раз: 687
время: 579
хороший: 518
работать: 497
знать: 477
делать: 445
деньга: 439
работа: 415
говорить: 371
два: 351
первый: 350
случай: 347
дело: 343
день: 341
новый: 339
стоить: 335
сделать: 316
сказать: 312


Ну с тональностью частотные списки выглядят очень даже неплохо

Делаю частотные списки по датасетам в общем, без деления по лэйблам

In [10]:
total_freq_sent = Counter()
for counter in label_freqs_sent.values():
    total_freq_sent.update(counter)

total_freq_tox = Counter()
for counter in label_freqs_tox.values():
    total_freq_tox.update(counter)
# Теперь total_freq — это единый Counter со всеми леммами из всех лэйблов
top_n = 100  # например, 100 самых частотных
print('Sent')
for lemma, freq in total_freq_sent.most_common(top_n):
    print(f"{lemma}: {freq}")
print('Tox')
for lemma, freq in total_freq_tox.most_common(top_n):
    print(f"{lemma}: {freq}")

Sent
фильм: 50827
человек: 11989
хороший: 9758
герой: 8251
жизнь: 7085
смотреть: 7030
актёр: 7020
главный: 6765
время: 6471
сказать: 6416
кино: 6172
первый: 6125
сюжет: 6101
картина: 5589
роль: 5343
игра: 5325
ни: 5324
раз: 5123
история: 4739
говорить: 4694
посмотреть: 4639
просмотр: 4563
год: 4470
зритель: 4362
режиссёр: 4201
знать: 4168
мир: 3984
конец: 3910
персонаж: 3848
видеть: 3630
сцена: 3624
момент: 3565
сделать: 3553
интересный: 3425
часть: 3398
любовь: 3241
понравиться: 3223
слово: 3215
стоить: 3215
два: 3072
дело: 3014
новый: 3005
снять: 2982
думать: 2964
работа: 2826
место: 2823
образ: 2809
ребёнок: 2724
увидеть: 2706
получиться: 2668
играть: 2619
делать: 2619
понять: 2587
друг: 2545
идея: 2538
общий: 2500
любить: 2473
второй: 2469
сыграть: 2463
показать: 2457
плохой: 2351
актёрский: 2311
понимать: 2265
смысл: 2224
книга: 2217
музыка: 2211
глаз: 2206
экран: 2186
каждый: 2183
сценарий: 2173
чувство: 2146
лицо: 2143
взгляд: 2133
последний: 2073
хотеться: 2050
смешной: 2016
ми

In [11]:
print(sum(total_freq_tox.values()))
print(sum(total_freq_sent.values()))

207014
1616936


Так как абсолютные частоты по датасетам сильно разнятся, посчитаем относительные

In [12]:
label_freqs_tox_per = {}
for k, v in label_freqs_tox.items():
  v = {key : value/sum(total_freq_tox.values()) for key, value in v.items()}
  v = v = {k: v for k, v in sorted(v.items(), key=lambda item: item[1], reverse=True)}
  label_freqs_tox_per[k] = v

label_freqs_sent_per = {}
for k, v in label_freqs_sent.items():
  v = {key : value/sum(total_freq_sent.values()) for key, value in v.items()}
  v = {k: v for k, v in sorted(v.items(), key=lambda item: item[1], reverse=True)}
  label_freqs_sent_per[k] = v

total_freq_tox_per = {}
for k, v in total_freq_tox.items():
    total_freq_tox_per[k] = v/sum(total_freq_tox.values())
total_freq_tox_per = {k: v for k, v in sorted(total_freq_tox_per.items(), key=lambda item: item[1], reverse=True)}

total_freq_sent_per = {}
for k, v in total_freq_sent.items():
    total_freq_sent_per[k] = v/sum(total_freq_tox.values())
total_freq_sent_per = {k: v for k, v in sorted(total_freq_sent_per.items(), key=lambda item: item[1], reverse=True)}

Функция для подсчета меры Шайкевича

Я решила попробовать убрать слово "фильм", потому что оно овер частотно в датасете для сентимента и портит картину. Заодно убрала еще три самых частотных везде, но это картину не поменяло все равно.

In [13]:
def compute_shaykevich_metric(corpus1_counts, corpus2_counts, top_n=1000):
    all_lemmas = set(corpus1_counts) | set(corpus2_counts)
    combined_freq = {
        lemma: corpus1_counts.get(lemma, 0) + corpus2_counts.get(lemma, 0)
        for lemma in all_lemmas
    }

    top_lemmas = sorted(combined_freq.items(), key=lambda x: x[1], reverse=True)[:top_n]
    top_words = [lemma for lemma, _ in top_lemmas ]

    top_words.remove('фильм')
    top_words.remove('человек')
    top_words.remove('хороший')
    top_words.remove('год')
    print(top_words)

    sum_min = 0
    sum_avg = 0
    for lemma in top_words:
        f1 = corpus1_counts.get(lemma, 0)
        f2 = corpus2_counts.get(lemma, 0)
        sum_min += min(f1, f2)
        sum_avg += (f1 + f2) / 2

    if sum_avg == 0:
        return 0.0

    return sum_min / sum_avg

Считаем меры для разных сочетаний: первые две - по датасетам в целом, дальще комбинируем датасеты по лэйблам. Например, sent_tox_bad_1 - берем список из сентимента по лэйблу Bad и список из токсичности по лэйблу 1.0

Для сравнения я решила игнорить нейтральные отзывы в сентименте, хотя можно было бы, например, объединить их с положительными, хотя не думаю, что это сильно бы что-то поменяло

In [14]:
sent_good_bad = compute_shaykevich_metric(label_freqs_sent_per['Good'], label_freqs_sent_per['Bad'])
tox_yes_no = compute_shaykevich_metric(label_freqs_tox_per[1.0], label_freqs_tox_per[0.0])
sent_tox_bad_1 = compute_shaykevich_metric(label_freqs_sent_per['Bad'], label_freqs_tox_per[1.0])
sent_tox_bad_0 = compute_shaykevich_metric(label_freqs_sent_per['Bad'], label_freqs_tox_per[0.0])
sent_tox_good_1 = compute_shaykevich_metric(label_freqs_sent_per['Good'], label_freqs_tox_per[1.0])
sent_tox_good_0 = compute_shaykevich_metric(label_freqs_sent_per['Good'], label_freqs_tox_per[0.0])
sent_tox_all = compute_shaykevich_metric(total_freq_sent_per, total_freq_tox_per)

['герой', 'актёр', 'смотреть', 'жизнь', 'главный', 'время', 'сказать', 'кино', 'сюжет', 'первый', 'картина', 'ни', 'игра', 'роль', 'раз', 'говорить', 'история', 'просмотр', 'посмотреть', 'зритель', 'знать', 'режиссёр', 'мир', 'персонаж', 'видеть', 'конец', 'сцена', 'сделать', 'слово', 'момент', 'любовь', 'часть', 'интересный', 'стоить', 'два', 'снять', 'думать', 'новый', 'дело', 'ребёнок', 'понравиться', 'работа', 'увидеть', 'делать', 'образ', 'играть', 'место', 'друг', 'понять', 'получиться', 'сыграть', 'любить', 'показать', 'актёрский', 'второй', 'плохой', 'понимать', 'общий', 'идея', 'сценарий', 'каждый', 'музыка', 'экран', 'смысл', 'чувство', 'минута', 'смешной', 'хотеться', 'глаз', 'лицо', 'книга', 'последний', 'юмор', 'взгляд', 'снимать', 'являться', 'шедевр', 'происходить', 'душа', 'деньга', 'создатель', 'полный', 'настолько', 'идти', 'вопрос', 'итог', 'день', 'совершенно', 'жить', 'мультфильм', 'красивый', 'кинематограф', 'сказка', 'шутка', 'впечатление', 'больший', 'эмоция', '

In [25]:
print(f"""tox_yes_no: {sent_good_bad}
sent_good_bad: {sent_good_bad}
sent_good_bad_1: {sent_tox_bad_1}
sent_tox_bad_0: {sent_tox_bad_0}
sent_tox_good_1: {sent_tox_good_1}
sent_tox_good_0: {sent_tox_good_0}
sent_tox_all: {sent_tox_all}"""
    )

tox_yes_no: 0.7941970264052487
sent_good_bad: 0.7941970264052487
sent_good_bad_1: 0.503091870382508
sent_tox_bad_0: 0.4959720588944889
sent_tox_good_1: 0.477793607294367
sent_tox_good_0: 0.4831549767627885
sent_tox_all: 0.1502995471231234


Уже итак видно, что токсичные и негативные тексты не более похожи между собой, чем все остальные, но посчитаем еще хи-квадрат:

In [35]:
chi2, p, dof, expected = chi2_contingency([[sent_tox_bad_1, sent_tox_bad_0],
                                           [sent_tox_good_1, sent_tox_good_0]])
print(f"χ² = {chi2:.5f}, p-value = {p:.5f}, degree of freedom = {dof}, expected frequencies = {expected}")

χ² = 0.00000, p-value = 1.00000, degree of freedom = 1, expected frequencies = [[0.49998012 0.49908381]
 [0.48090535 0.48004323]]


Попробуем убрать поправку Йейтса, чтобы у нас не занулялся хи-квадрат

In [33]:
chi2, p, dof, expected = chi2_contingency([[sent_tox_bad_1, sent_tox_bad_0],
                                           [sent_tox_good_1, sent_tox_good_0]], correction=False)
print(f"χ² = {chi2:.5f}, p-value = {p:.5f}, degree of freedom = {dof}, expected frequencies = {expected}")

χ² = 0.00008, p-value = 0.99291, degree of freedom = 1, expected frequencies = [[0.49998012 0.49908381]
 [0.48090535 0.48004323]]


На всякий случай посчитала еще через кастомную функцию

In [36]:
def chi_square(a, b, c, d):
    """Вычисляет хи-квадрат для 2x2 таблицы частот"""
    N = a + b + c + d
    E1 = (a + c) * (a + b) / N  # ожидаемое значение для ячейки a
    E2 = (a + c) * (c + d) / N  # ожидаемое значение для ячейки c

    chi2 = ((a - E1) ** 2) / E1 + ((c - E2) ** 2) / E2
    return chi2

a = sent_tox_bad_1
b = sent_tox_bad_0
c = sent_tox_good_1
d = sent_tox_good_0

chi2_value = chi_square(a, b, c, d)
print(f"χ² = {chi2_value:.7f}")

χ² = 0.0000395


Пороговое значнение хи-квадрата, после которого мы можем отвергнуть нулевую гипотезу для первой степени свободы равно 3.841. Наше значение хи-квадрата практически равно нулю, и естественно, нулевую гипотезу мы не отвергаем.

Я просчиталась, но где?

Во-первых, с датасетом по сентименту действительно что-то не так, но это я уже для курсовой буду смотреть. Во-вторых, я взяла один датасет тематический - отзывы на фильмы, а другой - просто комменты про все. Это ожидаемо сместило частотные списки в сторону лексики, связанной с кино. Ну и еще, я думаю, что можно было бы взять слова, частотные по всем лэйблам для каждого датасета и выкинуть их. Как, например, я сделала со словом "фильм".

Ну а на основе тех данных, которые есть, можно сделать вывод, что лексика в токсичных и негативных текстах не более близка, чем у всех остальных